In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle

from datetime import date, datetime, timedelta

In [ ]:
infilename = 'tabula-schedule-of-classes-3.csv'

#data = np.loadtxt(infilename,unpack=True,dtype=str,delimiter=',')
df = pd.read_csv(infilename)

In [ ]:
print(df.shape)
df.columns

In [ ]:
df = df.loc[df['COURSE'] != 'Course']
print(df.shape)

In [ ]:
df['COURSE']

names = []

for val in df['COURSE']:
    names.append(val[0:4])
#print(names)

departments = np.unique(names)
print(departments)

df['Department'] = names

In [ ]:
start_times = []
end_times = []
duration = []

for val in df['Time']:
    sdt = None
    edt = None
    offset = 0
    
    if val.find('-')<0 or val=='Arranged':
        start_times.append(datetime(2021,1,1,1,0))
        end_times.append(datetime(2021,1,1,2,0))
        duration.append(-1)
        continue
    
    s = val.split('-')[0]
    sh,sm = 0,0
    if s.find(':')>=0:
        sh,sm = s.split(':')
        sh = int(sh)
        if sm[-1]=='p':
            offset = 12
            if sh!=12:
                sh += offset
        sm = int(sm[0:2])

        
    e = val.split('-')[1]
    eh,em = 0,0
    if e.find(':')>=0:
        eh,em = e.split(':')
        eh = int(eh)
        em = int(em[0:2])
        
        if eh<sh:
            offset = 12

        if eh != 12:
            eh += offset
        
    #print(s,sh,sm)
    sdt = datetime(2021,1,1,sh,sm)
    edt = datetime(2021,1,1,eh,em)
    dur = edt-sdt
    
    start_times.append(sdt)
    end_times.append(edt)
    duration.append(dur.total_seconds()/3600.)

start_times = np.array(start_times)
end_times = np.array(end_times)
duration = np.array(duration)

df['Start'] = start_times
df['End'] = end_times
df['Duration'] = duration

In [ ]:
end_times[0],start_times[0]

In [ ]:
x = end_times - start_times

plt.figure()
plt.hist(duration,bins=8,range=(0,4))


In [ ]:
df[df['Department']=='PHYS']

In [ ]:
df[df['COURSE']=='MATH120']

In [ ]:
df[df['CRN']=='40151']

In [ ]:
days_dict = {'M':1, 'T':2, 'W':3, 'R':4, 'F':5}

#c = df[df['CRN']=='46792']
c = df[df['CRN']=='42697']


s = c['Start'].iloc[0]
dur = c['Duration'].iloc[0]

days = c['Days'].iloc[0]
name = c['COURSE'].iloc[0]

#print(days)
#print(type(days))
#days = days.split()
#print(days)

ytimes = []

for i in range(8,22):
    d = datetime(2021,1,1,i,0)
    ytimes.append(d)

    d = datetime(2021,1,1,i,30)
    ytimes.append(d)

#xtimes = ["M", "T", "W", "R", "F"]
xtimes = np.arange(1,6,1)

    
plt.figure(figsize=(12,8))
#plt.plot([min(xtimes)],[min(ytimes)],'o')
#plt.plot([max(xtimes)],[max(ytimes)],'o')

#plt.gca().add_patch(Rectangle((0.5,datetime(2021,1,1,17,0)), 1, timedelta(hours=3), alpha=0.4, facecolor='b'))

for day in days:
    day_num = days_dict[day]-0.5
    print(day_num)
    plt.gca().add_patch(Rectangle((day_num, s), 1, timedelta(hours=1), alpha=0.1, facecolor='b'))
    plt.text(day_num+0.2,s+timedelta(minutes=30),name)


#plt.plot(dates,yvals,'o');

# Play around with the following lines so you see what they do
#plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y/%m"))
plt.gca().yaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))

plt.xticks(rotation=30);
plt.xlim(min(xtimes)-1,max(xtimes)+1)
plt.ylim(min(ytimes),max(ytimes))
plt.gca().invert_yaxis()



In [ ]:
end_times[0].hour

In [ ]:
df['Start'][0].hour